<a href="https://colab.research.google.com/github/hauyinghk/BERT-CSV/blob/master/Submission_Predicting_Sentiment_with_BERT_on_TF_Hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Original source code reference from:
# https://github.com/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb
# Modified by Hor. A.Y. for the Dissertation

#Fine-tuning the BERT model with customerized dataset (data from LIHKG.COM)
#Predicting the Sentiment (positive / negative) with BERT on TF Hub


https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata1200.csv
https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata6000.csv

The dataset is prepared by Hor. A.Y. The number of entries in the datasets are 1200 and 6000 respectively with two columns namely "comment" and "sentiment".  "comment" is the content obtained from LIHKG.COM for sentiment analysis.  "sentiment" is assigned to "0" (Negative) or "1" (Positive)  for fine-tuning the BERT model.

In [2]:
%tensorflow_version 1.x
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
from datetime import datetime

TensorFlow 1.x selected.
1.15.2


The above standard libraries are imported, and make sure that the version of tensorflow is fit for the model. 

We also have to install bert-tensorflow for the sentiment analysis

In [3]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 3.0MB/s 


In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [5]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'output'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = '' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: output *****


#Data

In [6]:
#Get the testing data from github: testdata1200.csv
!wget http://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata1200.csv
test = pd.read_csv("testdata1200.csv" , error_bad_lines=False, sep=",") #, error_bad_lines=False, sep=',')

--2020-06-30 22:48:15--  http://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata1200.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata1200.csv [following]
--2020-06-30 22:48:15--  https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata1200.csv
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 179709 (175K) [text/plain]
Saving to: ‘testdata1200.csv’

testdata1200.csv    100%[===================>] 175.50K  --.-KB/s    in 0.02s   

2020-06-30 22:48:15 (7.35 MB/s) - ‘testdata1200.csv’ saved [179709/179709]



In [7]:
#Get the training data from github: testdata6000.csv
!wget http://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata6000.csv
train = pd.read_csv("testdata6000.csv" , error_bad_lines=False, sep=",") #, error_bad_lines=False, sep=',')

URL transformed to HTTPS due to an HSTS policy
--2020-06-30 22:48:17--  https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/testdata6000.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 590313 (576K) [text/plain]
Saving to: ‘testdata6000.csv’

testdata6000.csv    100%[===================>] 576.48K  --.-KB/s    in 0.04s   

2020-06-30 22:48:17 (12.6 MB/s) - ‘testdata6000.csv’ saved [590313/590313]



A sample of 6000 train and a sample of 1200 test, respectively.

In [8]:

train = train.sample(len(train))
train.head()

test = test.sample(len(test))
test.head()

#print the 5 rows of train
print(train.head())
print(test.head())

                                                comment  sentiment
5450                                       條友自己仆親又屈人打佢？          0
779   顯示更多?人見到我地講埋呢?野心諗你班人唔正路 走先可能一齊討論呢嚇走哂D絲打好賤呀根據連登...          0
645                                          #touch#dog          0
5386  外國足球打?完左 你去對家地頭慶祝 你唔死都一身殘啦 成班鄉下佬唔識分莊閒唔緊要 我地慢慢教...          1
4148  顯示更多有冇玩過lego呀細個鳩砌有我?度諗用AA膠 貼d Lego上牆到會唔會好難先抆到呢...          0
                                      comment  sentiment
326         push多謝手足支持 like左FB IG未呀\n#bye#lm2          1
535                                         推          1
248   大家要出力推\n#drum#pig\n#drum#pig\n#drum#pig          0
34                                       籌旗出聲          1
1165           #kill#\n#kill#\n#kill#\n#kill#          0


In [9]:
train.columns

Index(['comment', 'sentiment'], dtype='object')

Our input data is 'sentence' column and our label is 'sentiment' column (0, 1 for negative and positive, respecitvely)

In [10]:
DATA_COLUMN = 'comment'
LABEL_COLUMN = 'sentiment'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [11]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do the below 
(Included in the Python library):

1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [12]:
# This is a path to an uncased (all lowercase) version of BERT
#BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
BERT_MODEL_HUB = "https://storage.googleapis.com/tfhub-modules/google/bert_chinese_L-12_H-768_A-12/1.tar.gz" 

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


The BERT model expects lowercase data (Stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [13]:
# A test sentences to see if the tokenizer works properly
tokenizer.tokenize("I love this lacrosse match")

['[UNK]', 'love', 'this', 'la', '##cr', '##os', '##se', 'match']

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [14]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 6000


INFO:tensorflow:Writing example 0 of 6000


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 0 (id = 0)


INFO:tensorflow:label: 0 (id = 0)


#Creating a model

Now focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [15]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [16]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [17]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook
# (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [18]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [19]:
# Specify outpit directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [20]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f29cb6c6668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': 'output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f29cb6c6668>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [21]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Fine-turning the BERT Model. I am using a Colab notebook running on Google's GPUs, my training time was around 6 minutes 

In [22]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving checkpoints for 562 into output/model.ckpt.


INFO:tensorflow:Saving checkpoints for 562 into output/model.ckpt.


INFO:tensorflow:Loss for final step: 0.22296299.


INFO:tensorflow:Loss for final step: 0.22296299.


Training took time  0:05:39.363343


Now let's use our test data to see how well our model did:

In [23]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [24]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
/tensorflow-1.15.2/python3.6/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.


...

INFO:tensorflow:Saving dict for global step 562: auc = 0.9578231, eval_accuracy = 0.96330273, f1_score = 0.9694444, false_negatives = 8.0, false_positives = 36.0, global_step = 562, loss = 0.11938026, precision = 0.95095366, recall = 0.98866856, true_negatives = 457.0, true_positives = 698.0


INFO:tensorflow:Saving dict for global step 562: auc = 0.9578231, eval_accuracy = 0.96330273, f1_score = 0.9694444, false_negatives = 8.0, false_positives = 36.0, global_step = 562, loss = 0.11938026, precision = 0.95095366, recall = 0.98866856, true_negatives = 457.0, true_positives = 698.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 562: output/model.ckpt-562


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 562: output/model.ckpt-562


{'auc': 0.9578231,
 'eval_accuracy': 0.96330273,
 'f1_score': 0.9694444,
 'false_negatives': 8.0,
 'false_positives': 36.0,
 'global_step': 562,
 'loss': 0.11938026,
 'precision': 0.95095366,
 'recall': 0.98866856,
 'true_negatives': 457.0,
 'true_positives': 698.0}

Now let's write code to make predictions on new sentences:

In [25]:
def getPrediction(in_sentences):
  labels = ["Negative", "Positive"]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [26]:
#prepared a CSV with sentences for prediction
!wget https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/topred.csv    #The CSV File for predictions

import pandas as pd
df = pd.read_csv("topred.csv")                                                  #The CSV File for predictions
df.values.tolist()
print(df)
predictions = getPrediction(df)

--2020-06-30 22:54:52--  https://raw.githubusercontent.com/hauyinghk/BERT-CSV/master/topred.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 965 [text/plain]
Saving to: ‘topred.csv’

topred.csv          100%[===================>]     965  --.-KB/s    in 0s      

2020-06-30 22:54:52 (110 MB/s) - ‘topred.csv’ saved [965/965]

Empty DataFrame
Columns: [私了, 今日好成功, 要走, 兄弟爬山，各有各做, 煲底見, 打斷佢手腳, 無忘831, FREE?HK, 保護記者, 黑警死全家, 我今日好高興, 燒佢屋企將佢老婆仔女先姦後殺再慢慢虐待到佢生不如死, 唔知得唔得?狗成日用椒噴我?我地可以搵野噴返佢? 有睇過話用噴漆或者滅火筒, 要求3點回應撤回反送中條例否則行動升級!!!!, 同時間我地要不斷煽風點火，將本身屌一哥D post/留言不斷讚好, 最前線人員，須手持盾牌，幫手擋子彈全民扔雞蛋以及臭?野（例如屎水彈）, 眼罩 口罩 頭? 較剪 大粒電池  以上5樣, 龍和道準備開波 快手, 好多地方都有得拎免費報紙 易燃 可做性又大 千其唔好拎, 汽油彈汽油 番梘比例幾多至好, 用無人機拖d油漆去淋9佢, 先用火攻 將汽油彈轟落黑警到 再突擊用鋁熱劑係車頂開窿, 要攻破就簡單啦！ 扔D油漆既野 遮佢地個view同cam 即時行唔到, Unnamed: 23]
Index: [

INFO:tensorflow:Writing example 0 of 24


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: 


...

INFO:tensorflow:Restoring parameters from output/model.ckpt-562


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


Okay! Let see the prediction results of the fine tuned sentiment classifier!

In [27]:
predictions

[('私了', array([-0.03024983, -3.513353  ], dtype=float32), 'Negative'),
 ('今日好成功', array([-4.9754696 , -0.00692918], dtype=float32), 'Positive'),
 ('要走', array([-0.0723919, -2.6616387], dtype=float32), 'Negative'),
 ('兄弟爬山，各有各做', array([-2.6032016 , -0.07692013], dtype=float32), 'Positive'),
 ('煲底見', array([-0.01760452, -4.04839   ], dtype=float32), 'Negative'),
 ('打斷佢手腳', array([-0.03563511, -3.3521895 ], dtype=float32), 'Negative'),
 ('無忘831', array([-0.01452938, -4.23884   ], dtype=float32), 'Negative'),
 ('FREE?HK', array([-4.6129584 , -0.00997192], dtype=float32), 'Positive'),
 ('保護記者', array([-0.02543025, -3.6845045 ], dtype=float32), 'Negative'),
 ('黑警死全家', array([-0.02383438, -3.7485192 ], dtype=float32), 'Negative'),
 ('我今日好高興', array([-4.3732038 , -0.01269097], dtype=float32), 'Positive'),
 ('燒佢屋企將佢老婆仔女先姦後殺再慢慢虐待到佢生不如死',
  array([-0.02248193, -3.8062656 ], dtype=float32),
  'Negative'),
 ('唔知得唔得?狗成日用椒噴我?我地可以搵野噴返佢? 有睇過話用噴漆或者滅火筒',
  array([-0.02665384, -3.6381183 ], dtype=float32